# ChatGPT API in ython
This notebook aims to provide a simple example of interacting with ChatGPT  using its python API

In [1]:
# Install Open AI API package
!pip install openai -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.1/267.1 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.6 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/drive', force_remount=True)

Mounted at /drive


In [14]:
ls /drive/MyDrive/'Colab Notebooks'/openai.api_key

ls: cannot access '/drive/MyDrive/Colab Notebooks/openai.api_key': No such file or directory


In [30]:
import openai
import os.path
import pandas as pd

if os.path.isfile("/drive/MyDrive/Colab Notebooks/openai.api_key"):
    openai.api_key = pd.read_csv("/drive/MyDrive/Colab Notebooks/openai.api_key", header=None).loc[0,0]
    print("Your open AI API key was obtained from /drive/MyDrive/'Colab Notebooks'/openai.api_key")
else:
    from getpass import getpass
    secret = getpass('Enter the openai passcode value (can be obtained from https://platform.openai.com/account/api-keys): ')
    openai.api_key = secret
    !echo {secret} > openai.api_key
    !cp openai.api_key /drive/MyDrive/'Colab Notebooks'/openai.api_key
    print("Your open AI API key was copied to /drive/MyDrive/Colab Notebooks/openai.api_key")

Enter the openai passcode value (can be obtained from https://platform.openai.com/account/api-keys): ··········
Your open AI API key was copied to /drive/MyDrive/Colab Notebooks/openai.api_key


In [31]:
client = openai.OpenAI(api_key=openai.api_key)

messages=[{"role": "user", "content": "Turn this movie title into emojis: Harry Potter"}]

responses = client.chat.completions.create(
    model="gpt-4",
   messages = messages
    )

response = responses.choices[0].message.content
print(response)

🧙‍♂️👦🏻⚡🔮📚


## Bringing a list of highly rated movies from MovieLens
Data source: [MovieLens 25M dataset](https://grouplens.org/datasets/movielens/)

25 million ratings - 62,000 movies by 162,000 users - Released 12/2019

In [32]:
# Download movie 25M lens dataset
!wget https://files.grouplens.org/datasets/movielens/ml-25m.zip

--2024-04-06 04:38:58--  https://files.grouplens.org/datasets/movielens/ml-25m.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 261978986 (250M) [application/zip]
Saving to: ‘ml-25m.zip’

ml-25m.zip          100%[===================>] 249.84M  17.5MB/s    in 16s     

2024-04-06 04:39:15 (16.1 MB/s) - ‘ml-25m.zip’ saved [261978986/261978986]



In [33]:
# Unzip ml-25m.zip
!unzip ml-25m.zip

Archive:  ml-25m.zip
   creating: ml-25m/
  inflating: ml-25m/tags.csv         
  inflating: ml-25m/links.csv        
  inflating: ml-25m/README.txt       
  inflating: ml-25m/ratings.csv      
  inflating: ml-25m/genome-tags.csv  
  inflating: ml-25m/genome-scores.csv  
  inflating: ml-25m/movies.csv       


In [34]:
ls -lh ml-25m/

total 1.1G
-rw-rw-r-- 1 root root 416M Nov 21  2019 genome-scores.csv
-rw-rw-r-- 1 root root  18K Nov 21  2019 genome-tags.csv
-rw-rw-r-- 1 root root 1.4M Nov 21  2019 links.csv
-rw-rw-r-- 1 root root 2.9M Nov 21  2019 movies.csv
-rw-rw-r-- 1 root root 647M Nov 21  2019 ratings.csv
-rw-rw-r-- 1 root root  11K Nov 21  2019 README.txt
-rw-rw-r-- 1 root root  38M Nov 21  2019 tags.csv


In [35]:
movies = pd.read_csv('ml-25m/movies.csv', encoding='latin-1') # encoding='latin-1' was used
ratings = pd.read_csv('ml-25m/ratings.csv') # tab separated data

ratings = ratings.groupby('movieId').agg({'rating': 'mean', 'userId': 'count'})
ratings.columns = ['avg_rating', 'n_rating']
ratings = movies[['movieId', 'title']].merge(ratings[ratings.n_rating >= 10000].reset_index(), on='movieId').sort_values('avg_rating', ascending=False)
ratings = ratings.drop('movieId', axis=1).reset_index(drop=True)
ratings.head(10)

,title,avg_rating,n_rating
0,"Shawshank Redemption, The (1994)",4.413576,81482
1,"Godfather, The (1972)",4.324336,52498
2,"Usual Suspects, The (1995)",4.284353,55366
3,"Godfather: Part II, The (1974)",4.261759,34188
4,Seven Samurai (Shichinin no samurai) (1954),4.254769,13367
5,Schindler's List (1993),4.247579,60411
6,12 Angry Men (1957),4.243014,16569
7,Rear Window (1954),4.237948,20162
8,Fight Club (1999),4.228311,58773
9,One Flew Over the Cuckoo's Nest (1975),4.218662,36058


In [38]:
def GPTCompletion(messages, max_tokens):
    responses = client.chat.completions.create(
        model="gpt-4",
        max_tokens=max_tokens,
        temperature=.5,  # What sampling temperature to use, between 0 and 2. Higher values like 0.8 will make the output more random, while lower values like 0.2 will make it more focused and deterministic. We generally recommend altering this or top_p but not both.
        messages = messages
        )
    response = responses.choices[0].message.content

    return response

In [39]:
pd.set_option('display.max_colwidth', None)  # to show the full text in summary
enoji_df = pd.DataFrame()

for movie in ratings.title[:10]:
    messages=[{"role": "user", "content": "Turn this movie title into emojis using the movie script and not just the title: {}".format(movie)}]
    emoji_response = GPTCompletion(messages, max_tokens=20)

    messages=[{"role": "user", "content": "Provide a one line summary about the the movie title: {}. Max 20 words".format(movie)}]
    summary_response = GPTCompletion(messages, max_tokens=100)

    enoji_df = pd.concat([enoji_df, pd.DataFrame({'movie': [movie], 'emoji': [emoji_response], 'summary': [summary_response]})], ignore_index=True)

enoji_df

,movie,emoji,summary
0,"Shawshank Redemption, The (1994)",👨‍⚖️🔒📖🙏,"A wrongly imprisoned man forms a remarkable friendship and finds solace through hope in a brutal, corrupt prison."
1,"Godfather, The (1972)",👴🙏🔫🇮🇹🍊💼,An organized crime family's patriarch transfers control of his empire to his reluctant son.
2,"Usual Suspects, The (1995)",👥🔎🕵️‍♂️🚢💥,A survivor tells a twisted tale of deception and crime involving five criminals who meet in a random police lineup.
3,"Godfather: Part II, The (1974)",🙏👨‍👧‍👦🕵️,The movie chronicles the early life of Vito Corleone and his son Michael's struggle to expand and protect his empire.
4,Seven Samurai (Shichinin no samurai) (1954),7️⃣👥⚔️🇯🇵,A group of samurais protect a village from bandits in 16th century Japan in this Akira Kurosawa-directed film.
5,Schindler's List (1993),🇩🇪📝📃🔎🕍,A German businessman saves over a thousand Polish Jews during the Holocaust by employing them in his factories.
6,12 Angry Men (1957),1️⃣2️⃣😡👨‍⚖,"A jury debates a teenager's guilt in a murder case, with one juror advocating for reasonable doubt."
7,Rear Window (1954),🔙🪟👀🕵️‍♂️,A wheelchair-bound photographer spies on his neighbors and becomes convinced one of them has committed murder.
8,Fight Club (1999),👊💥👥🚬🧼🌃🕰,"An insomniac office worker forms an underground fight club with a soap salesman, leading to a complex psychological journey."
9,One Flew Over the Cuckoo's Nest (1975),👤🕊️✈️➡️🐦🏠,A rebellious patient inspires fellow inmates at a mental institution to challenge the oppressive regime of the head nurse.
